In [0]:
%scala
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._

val session = SparkSession.builder.appName("M&Ms").getOrCreate()

val file = "/FileStore/shared_uploads/giovanni.rodriguez@bosonit.com/mnm_dataset.csv"

import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
session: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@1cafa884
file: String = /FileStore/shared_uploads/giovanni.rodriguez@bosonit.com/mnm_dataset.csv

In [0]:
%scala
val dataFrame = session.read.format("csv").option("header", "true").option("inferSchema", "true").load(file)
dataFrame.persist()

dataFrame: org.apache.spark.sql.DataFrame = [State: string, Color: string ... 1 more field]
res0: dataFrame.type = [State: string, Color: string ... 1 more field]

In [0]:
%scala
val countDataFrame = dataFrame.select("State", "Color", "Count").groupBy("State", "Color").agg(count("Count").alias("Total")).orderBy(desc("Total"))
countDataFrame.show()

+-----+------+-----+
State| Color|Total|
+-----+------+-----+
 CA|Yellow| 1807|
 WA| Green| 1779|
 OR|Orange| 1743|
 TX| Green| 1737|
 TX| Red| 1725|
 CA| Green| 1723|
 CO|Yellow| 1721|
 CA| Brown| 1718|
 CO| Green| 1713|
 NV|Orange| 1712|
 TX|Yellow| 1703|
 NV| Green| 1698|
 AZ| Brown| 1698|
 WY| Green| 1695|
 CO| Blue| 1695|
 NM| Red| 1690|
 AZ|Orange| 1689|
 NM|Yellow| 1688|
 NM| Brown| 1687|
 UT|Orange| 1684|
+-----+------+-----+
only showing top 20 rows

countDataFrame: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [State: string, Color: string ... 1 more field]

In [0]:
%scala
val avgColorDataFrame = dataFrame.select("State", "Color", "Count").groupBy("State", "Color").agg(count("Count").alias("Total")).groupBy("Color").agg(avg("Total").alias("Average")).orderBy(asc("Average"))
avgColorDataFrame.show()

+------+-------+
 Color|Average|
+------+-------+
 Blue| 1644.9|
 Brown| 1651.0|
 Red| 1661.9|
Orange| 1669.7|
Yellow| 1679.6|
 Green| 1692.8|
+------+-------+

avgColorDataFrame: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Color: string, Average: double]

In [0]:
%scala
val vowelCountDataFrame = dataFrame.select("State", "Count").where(col("State").rlike("^[AEIOU]")).groupBy("State").agg(count("Count").alias("Total"))
vowelCountDataFrame.show()

+-----+-----+
State|Total|
+-----+-----+
 AZ|10001|
 OR| 9903|
 UT| 9886|
+-----+-----+

vowelCountDataFrame: org.apache.spark.sql.DataFrame = [State: string, Total: bigint]

In [0]:
%scala
val statsDataFrame = dataFrame.select("State", "Color", "Count").groupBy("State", "Color").agg(count("Count").alias("Total")).groupBy("Color").agg(min("Total").alias("Min"), max("Total").alias("Max"), avg("Total").alias("Average"), sum("Total").alias("Sum"))
statsDataFrame.show()

+------+----+----+-------+-----+
 Color| Min| Max|Average| Sum|
+------+----+----+-------+-----+
Orange|1595|1743| 1669.7|16697|
 Green|1591|1779| 1692.8|16928|
 Blue|1603|1695| 1644.9|16449|
 Brown|1532|1718| 1651.0|16510|
Yellow|1614|1807| 1679.6|16796|
 Red|1610|1725| 1661.9|16619|
+------+----+----+-------+-----+

statsDataFrame: org.apache.spark.sql.DataFrame = [Color: string, Min: bigint ... 3 more fields]

In [0]:
%scala
statsDataFrame.createOrReplaceTempView("stats")
session.table("stats").select("*").show()

+------+----+----+-------+-----+
 Color| Min| Max|Average| Sum|
+------+----+----+-------+-----+
Orange|1595|1743| 1669.7|16697|
 Green|1591|1779| 1692.8|16928|
 Blue|1603|1695| 1644.9|16449|
 Brown|1532|1718| 1651.0|16510|
Yellow|1614|1807| 1679.6|16796|
 Red|1610|1725| 1661.9|16619|
+------+----+----+-------+-----+

In [0]:
%scala
session.stop()